In [15]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol

def parse_pdb(file_path):
    protein_atoms = []
    ligand_atoms = []

    amino_acids = {}
    current_chain = None

    with open(file_path, 'r') as pdb_file:
        for line in pdb_file:
            if line.startswith("ATOM"):
                atom_type = line[12:16].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                chain = line[21]
                residue_number = int(line[22:26])
                residue_name = line[17:20]

                if chain != current_chain:
                    amino_acids = {}
                    current_chain = chain

                if chain not in amino_acids:
                    amino_acids[chain] = {}

                amino_acids[chain][residue_number] = residue_name

                protein_atoms.append((atom_type, x, y, z, chain, residue_number))

            elif line.startswith("HETATM"):
                atom_type = line[12:16].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])

                ligand_atoms.append((atom_type, x, y, z))

    #print("Parsed protein atoms:", len(protein_atoms))
    #print("Parsed ligand atoms:", len(ligand_atoms))
    #print("Parsed amino acids:", amino_acids)
    #print(ligand_atoms)
    
    return protein_atoms, ligand_atoms, amino_acids


def calculate_distances(protein_atoms, ligand_atoms):
    distances = []

    for protein_atom in protein_atoms:
       # print("Example protein atom:", protein_atom)  # Debug print
        for ligand_atom in ligand_atoms:
           # print("Example ligand atom:", ligand_atom)  # Debug print
            dist = np.sqrt((protein_atom[1] - ligand_atom[1]) ** 2 +
                           (protein_atom[2] - ligand_atom[2]) ** 2 +
                           (protein_atom[3] - ligand_atom[3]) ** 2)
            distances.append((protein_atom[0], ligand_atom[0], dist, protein_atom[4], protein_atom[5]))

    return distances


def sort_distances(distances):
    return sorted(distances, key=lambda x: x[2])
    
def write_to_excel(sorted_distances, output_file='distances.xlsx'):
    df = pd.DataFrame(sorted_distances, columns=['Protein Atom', 'Ligand Atom', 'Distance', 'Chain', 'Residue Number'])
    df.to_excel(output_file, index=False)
    print(f"Distances written to {output_file}")

def find_hydrogen_bonds(sorted_distances):
    hydrogen_bonds = []

    for protein_atom_type, ligand_atom_type, dist, chain, residue_number in sorted_distances:
        if (protein_atom_type[0] in ['N', 'O', 'S'] and ligand_atom_type[0] in ['N', 'O', 'S']) or \
                (ligand_atom_type[0] in ['N', 'O', 'S'] and protein_atom_type[0] in ['N', 'O', 'S']):
            if 2.7 <= dist <= 3.3:  # Adjust the threshold distance for hydrogen bonds
                hydrogen_bonds.append((chain, residue_number, protein_atom_type, ligand_atom_type, dist))

    return hydrogen_bonds

    if hydrogen_bonds:
        print("\nPossible Hydrogen Bond Interactions:")
        for bond in hydrogen_bonds:
            print(bond)
    else:
        print("\nNo possible Hydrogen Bond Interactions found.")

    return hydrogen_bonds




def visualize_ligand_sticks_with_labels(ligand_atoms):
    # Create a Py3Dmol view
    view = py3Dmol.view(width=800, height=600)

    # Create a PDB file string
    pdb_string = ""
    hydrogen_atoms = []  # Store hydrogen atoms separately for visualization
    for i, (atom_type, x, y, z) in enumerate(ligand_atoms):
        pdb_string += f"ATOM  {i+1:<5} {atom_type:<4} MOL     1    {x:>8.3f}{y:>8.3f}{z:>8.3f}\n"
        if atom_type.startswith('H'):  # Check if the atom is a hydrogen atom
            hydrogen_atoms.append((atom_type, x, y, z))  # Store hydrogen atoms

    # Add ligand atoms from the PDB file string
    view.addModel(pdb_string, "pdb")

    # Set style to stick representation
    view.setStyle({'stick': {}})

    # Add labels to ligand atoms
    for i, (atom_type, x, y, z) in enumerate(ligand_atoms):
        view.addLabel(atom_type, {'position': {'x': x, 'y': y, 'z': z}})

    # Add light blue spheres for hydrogen atoms
    for i, (atom_type, x, y, z) in enumerate(hydrogen_atoms):
        view.addSphere({'center': {'x': x, 'y': y, 'z': z}, 'radius': 0.3, 'color': 'lightblue'})

    # Set background color to black
    view.setBackgroundColor('black')

    # Zoom to fit the structure
    view.zoomTo()

    # Show the 3D view
    view.show()



pdb_file_path = "E:\\THESIS\\BACK_END\\4MS4-20240302T114112Z-001\\4MS4\\1.pdb"

protein_atoms, ligand_atoms, amino_acids = parse_pdb(pdb_file_path)
distances = calculate_distances(protein_atoms, ligand_atoms)
sorted_distances = sort_distances(distances)


# Write sorted distances to an Excel file
write_to_excel(sorted_distances, output_file='sorted_distances.xlsx')


# Finding and printing possible hydrogen bonds
hydrogen_bonds = find_hydrogen_bonds(sorted_distances)
if hydrogen_bonds:
    print("\nPossible Hydrogen Bond Interactions:")
    for chain, residue_number, protein_atom_type, ligand_atom_type, dist in hydrogen_bonds:
        amino_acid = amino_acids.get(chain, {}).get(residue_number, "Unknown")
        print(f"{amino_acid} {residue_number}({chain}){protein_atom_type}(donor) ------ {ligand_atom_type}(acceptor)   {dist}")
else:
    print("\nNo possible Hydrogen Bond Interactions found.")


visualize_ligand_sticks_with_labels(ligand_atoms)


Distances written to sorted_distances.xlsx

Possible Hydrogen Bond Interactions:
GLY 151(A)O(donor) ------ N1(acceptor)   2.718914673173837
SER 153(A)N(donor) ------ O2(acceptor)   3.078475109530692
SER 153(A)OG(donor) ------ O2(acceptor)   3.162595611202927


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [24]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol

def parse_pdb(file_path):
    protein_atoms = []
    ligand_atoms = []

    amino_acids = {}
    current_chain = None

    with open(file_path, 'r') as pdb_file:
        for line in pdb_file:
            if line.startswith("ATOM"):
                atom_type = line[12:16].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                chain = line[21]
                residue_number = int(line[22:26])
                residue_name = line[17:20]

                if chain != current_chain:
                    amino_acids = {}
                    current_chain = chain

                if chain not in amino_acids:
                    amino_acids[chain] = {}

                amino_acids[chain][residue_number] = residue_name

                protein_atoms.append((atom_type, x, y, z, chain, residue_number))

            elif line.startswith("HETATM"):
                atom_type = line[12:16].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])

                ligand_atoms.append((atom_type, x, y, z))

    #print("Parsed protein atoms:", len(protein_atoms))
    #print("Parsed ligand atoms:", len(ligand_atoms))
    #print("Parsed amino acids:", amino_acids)
    #print(ligand_atoms)
    
    return protein_atoms, ligand_atoms, amino_acids


def calculate_distances(protein_atoms, ligand_atoms):
    distances = []

    for protein_atom in protein_atoms:
       # print("Example protein atom:", protein_atom)  # Debug print
        for ligand_atom in ligand_atoms:
           # print("Example ligand atom:", ligand_atom)  # Debug print
            dist = np.sqrt((protein_atom[1] - ligand_atom[1]) ** 2 +
                           (protein_atom[2] - ligand_atom[2]) ** 2 +
                           (protein_atom[3] - ligand_atom[3]) ** 2)
            distances.append((protein_atom[0], ligand_atom[0], dist, protein_atom[4], protein_atom[5]))

    return distances


def sort_distances(distances):
    return sorted(distances, key=lambda x: x[2])
    
def write_to_excel(sorted_distances, output_file='distances.xlsx'):
    df = pd.DataFrame(sorted_distances, columns=['Protein Atom', 'Ligand Atom', 'Distance', 'Chain', 'Residue Number'])
    df.to_excel(output_file, index=False)
    print(f"Distances written to {output_file}")

def find_hydrogen_bonds(sorted_distances):
    hydrogen_bonds = []

    for protein_atom_type, ligand_atom_type, dist, chain, residue_number in sorted_distances:
        if (protein_atom_type[0] in ['N', 'O', 'S'] and ligand_atom_type[0] in ['N', 'O', 'S']) or \
                (ligand_atom_type[0] in ['N', 'O', 'S'] and protein_atom_type[0] in ['N', 'O', 'S']):
            if 2.7 <= dist <= 3.3:  # Adjust the threshold distance for hydrogen bonds
                hydrogen_bonds.append((chain, residue_number, protein_atom_type, ligand_atom_type, dist))

    return hydrogen_bonds

    if hydrogen_bonds:
        print("\nPossible Hydrogen Bond Interactions:")
        for bond in hydrogen_bonds:
            print(bond)
    else:
        print("\nNo possible Hydrogen Bond Interactions found.")

    return hydrogen_bonds




def visualize_ligand_sticks_with_labels(ligand_atoms):
    # Create a Py3Dmol view
    view = py3Dmol.view(width=800, height=600)

    # Create a PDB file string
    pdb_string = ""
    hydrogen_atoms = []  # Store hydrogen atoms separately for visualization
    for i, (atom_type, x, y, z) in enumerate(ligand_atoms):
        pdb_string += f"ATOM  {i+1:<5} {atom_type:<4} MOL     1    {x:>8.3f}{y:>8.3f}{z:>8.3f}\n"
        if atom_type.startswith('H'):  # Check if the atom is a hydrogen atom
            hydrogen_atoms.append((atom_type, x, y, z))  # Store hydrogen atoms

    # Add ligand atoms from the PDB file string
    view.addModel(pdb_string, "pdb")

    # Set style to stick representation
    view.setStyle({'stick': {}})

    # Add labels to ligand atoms
    for i, (atom_type, x, y, z) in enumerate(ligand_atoms):
        view.addLabel(atom_type, {'position': {'x': x, 'y': y, 'z': z}})

    # Add light blue spheres for hydrogen atoms
    for i, (atom_type, x, y, z) in enumerate(hydrogen_atoms):
        view.addSphere({'center': {'x': x, 'y': y, 'z': z}, 'radius': 0.3, 'color': 'lightblue'})

     # Add protein atoms as red spheres and label them
    for chain, residue_number, protein_atom_type, ligand_atom_type, dist in hydrogen_bonds:
        amino_acid = amino_acids.get(chain, {}).get(residue_number, "Unknown")
        # Find the coordinates of the protein atom involved in the hydrogen bond
        protein_atom_coords = next((x, y, z) for _, x, y, z, ch, rn in protein_atoms if ch == chain and rn == residue_number)
        # Add red sphere for the protein atom
        view.addSphere({'center': {'x': protein_atom_coords[0], 'y': protein_atom_coords[1], 'z': protein_atom_coords[2]}, 'radius': 0.3, 'color': 'red'})
        # Label the protein atom with amino acid and residue number
        label_text = f"{amino_acid} {residue_number}({chain})"
        view.addLabel(label_text, {'position': {'x': protein_atom_coords[0], 'y': protein_atom_coords[1], 'z': protein_atom_coords[2]}})


    # Set background color to black
    view.setBackgroundColor('black')

    # Zoom to fit the structure
    view.zoomTo()

    # Show the 3D view
    view.show()



pdb_file_path = "E:\\THESIS\\BACK_END\\4MS4-20240302T114112Z-001\\4MS4\\1.pdb"

protein_atoms, ligand_atoms, amino_acids = parse_pdb(pdb_file_path)
distances = calculate_distances(protein_atoms, ligand_atoms)
sorted_distances = sort_distances(distances)


# Write sorted distances to an Excel file
write_to_excel(sorted_distances, output_file='sorted_distances.xlsx')


# Finding and printing possible hydrogen bonds
hydrogen_bonds = find_hydrogen_bonds(sorted_distances)
if hydrogen_bonds:
    print("\nPossible Hydrogen Bond Interactions:")
    for chain, residue_number, protein_atom_type, ligand_atom_type, dist in hydrogen_bonds:
        amino_acid = amino_acids.get(chain, {}).get(residue_number, "Unknown")
        print(f"{amino_acid} {residue_number}({chain}){protein_atom_type}(donor) ------ {ligand_atom_type}(acceptor)   {dist}")
else:
    print("\nNo possible Hydrogen Bond Interactions found.")


visualize_ligand_sticks_with_labels(ligand_atoms)


Distances written to sorted_distances.xlsx

Possible Hydrogen Bond Interactions:
GLY 151(A)O(donor) ------ N1(acceptor)   2.718914673173837
SER 153(A)N(donor) ------ O2(acceptor)   3.078475109530692
SER 153(A)OG(donor) ------ O2(acceptor)   3.162595611202927


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [19]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol

def parse_pdb(file_path):
    protein_atoms = []
    ligand_atoms = []

    amino_acids = {}
    current_chain = None

    with open(file_path, 'r') as pdb_file:
        for line in pdb_file:
            if line.startswith("ATOM"):
                atom_type = line[12:16].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                chain = line[21]
                residue_number = int(line[22:26])
                residue_name = line[17:20]

                if chain != current_chain:
                    amino_acids = {}
                    current_chain = chain

                if chain not in amino_acids:
                    amino_acids[chain] = {}

                amino_acids[chain][residue_number] = residue_name

                protein_atoms.append((atom_type, x, y, z, chain, residue_number))

            elif line.startswith("HETATM"):
                atom_type = line[12:16].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])

                ligand_atoms.append((atom_type, x, y, z))

    #print("Parsed protein atoms:", len(protein_atoms))
    #print("Parsed ligand atoms:", len(ligand_atoms))
    #print("Parsed amino acids:", amino_acids)
    #print(ligand_atoms)
    
    return protein_atoms, ligand_atoms, amino_acids


def calculate_distances(protein_atoms, ligand_atoms):
    distances = []

    for protein_atom in protein_atoms:
       # print("Example protein atom:", protein_atom)  # Debug print
        for ligand_atom in ligand_atoms:
           # print("Example ligand atom:", ligand_atom)  # Debug print
            dist = np.sqrt((protein_atom[1] - ligand_atom[1]) ** 2 +
                           (protein_atom[2] - ligand_atom[2]) ** 2 +
                           (protein_atom[3] - ligand_atom[3]) ** 2)
            distances.append((protein_atom[0], ligand_atom[0], dist, protein_atom[4], protein_atom[5]))

    return distances


def sort_distances(distances):
    return sorted(distances, key=lambda x: x[2])
    
def write_to_excel(sorted_distances, output_file='distances.xlsx'):
    df = pd.DataFrame(sorted_distances, columns=['Protein Atom', 'Ligand Atom', 'Distance', 'Chain', 'Residue Number'])
    df.to_excel(output_file, index=False)
    print(f"Distances written to {output_file}")

def find_hydrogen_bonds(sorted_distances):
    hydrogen_bonds = []

    for protein_atom_type, ligand_atom_type, dist, chain, residue_number in sorted_distances:
        if (protein_atom_type[0] in ['N', 'O', 'S'] and ligand_atom_type[0] in ['N', 'O', 'S']) or \
                (ligand_atom_type[0] in ['N', 'O', 'S'] and protein_atom_type[0] in ['N', 'O', 'S']):
            if 2.7 <= dist <= 3.3:  # Adjust the threshold distance for hydrogen bonds
                hydrogen_bonds.append((chain, residue_number, protein_atom_type, ligand_atom_type, dist))

    return hydrogen_bonds

    if hydrogen_bonds:
        print("\nPossible Hydrogen Bond Interactions:")
        for bond in hydrogen_bonds:
            print(bond)
    else:
        print("\nNo possible Hydrogen Bond Interactions found.")

    return hydrogen_bonds




def visualize_ligand_sticks_with_labels(ligand_atoms, hydrogen_bonds, protein_atoms, amino_acids):
    # Create a Py3Dmol view
    view = py3Dmol.view(width=800, height=600)

    # Create a PDB file string for ligand atoms
    ligand_pdb_string = ""
    for i, (atom_type, x, y, z) in enumerate(ligand_atoms):
        ligand_pdb_string += f"ATOM  {i+1:<5} {atom_type:<4} LIG     1    {x:>8.3f}{y:>8.3f}{z:>8.3f}\n"

    # Add ligand atoms to the view
    view.addModel(ligand_pdb_string, "pdb")

    # Set style to stick representation for ligand atoms
    view.setStyle({'model': 0}, {'stick': {}})

    # Add labels to ligand atoms
    for i, (atom_type, x, y, z) in enumerate(ligand_atoms):
        view.addLabel(atom_type, {'position': {'x': x, 'y': y, 'z': z}})

    # Create a PDB file string for protein atoms
    protein_pdb_string = ""
    for i, (atom_type, x, y, z, chain, residue_number) in enumerate(protein_atoms):
        protein_pdb_string += f"ATOM  {i+1:<5} {atom_type:<4} PRO     1    {x:>8.3f}{y:>8.3f}{z:>8.3f}\n"

    # Add protein atoms to the view
    view.addModel(protein_pdb_string, "pdb")

    # Set style to sphere representation for protein atoms
    view.setStyle({'model': 1}, {'sphere': {'color': 'red'}})

    # Add labels to protein atoms
    for chain, residue_number, _, _, _ in hydrogen_bonds:
        amino_acid = amino_acids.get(chain, {}).get(residue_number, "Unknown")
        # Find the coordinates of the protein atom involved in the hydrogen bond
        protein_atom_coords = next((x, y, z) for _, x, y, z, ch, rn in protein_atoms if ch == chain and rn == residue_number)
        # Label the protein atom with amino acid and residue number
        label_text = f"{amino_acid} {residue_number}({chain})"
        view.addLabel(label_text, {'position': {'x': protein_atom_coords[0], 'y': protein_atom_coords[1], 'z': protein_atom_coords[2]}})

        # Find the ligand atom coordinates involved in the hydrogen bond
        ligand_atom_coords = None
        for _, x, y, z in ligand_atoms:
            if _ == "N1" or _ == "O2":
                ligand_atom_coords = (x, y, z)
                break

        # Connect protein atom to ligand with yellow dotted line
        view.addLine({'start': {'x': protein_atom_coords[0], 'y': protein_atom_coords[1], 'z': protein_atom_coords[2]},
                      'end': {'x': ligand_atom_coords[0], 'y': ligand_atom_coords[1], 'z': ligand_atom_coords[2]},
                      'color': 'yellow', 'dashed': True})

    # Set background color to black
    view.setBackgroundColor('black')

    # Zoom to fit the structure
    view.zoomTo()

    # Show the 3D view
    view.show()

pdb_file_path = "E:\\THESIS\\BACK_END\\4MS4-20240302T114112Z-001\\4MS4\\1.pdb"

protein_atoms, ligand_atoms, amino_acids = parse_pdb(pdb_file_path)
distances = calculate_distances(protein_atoms, ligand_atoms)
sorted_distances = sort_distances(distances)


# Write sorted distances to an Excel file
write_to_excel(sorted_distances, output_file='sorted_distances.xlsx')


# Finding and printing possible hydrogen bonds
hydrogen_bonds = find_hydrogen_bonds(sorted_distances)
if hydrogen_bonds:
    print("\nPossible Hydrogen Bond Interactions:")
    for chain, residue_number, protein_atom_type, ligand_atom_type, dist in hydrogen_bonds:
        amino_acid = amino_acids.get(chain, {}).get(residue_number, "Unknown")
        print(f"{amino_acid} {residue_number}({chain}){protein_atom_type}(donor) ------ {ligand_atom_type}(acceptor)   {dist}")
else:
    print("\nNo possible Hydrogen Bond Interactions found.")


# Call visualize_ligand_sticks_with_labels with ligand_atoms, hydrogen_bonds, protein_atoms, and amino_acids
visualize_ligand_sticks_with_labels(ligand_atoms, hydrogen_bonds, protein_atoms, amino_acids)


Distances written to sorted_distances.xlsx

Possible Hydrogen Bond Interactions:
GLY 151(A)O(donor) ------ N1(acceptor)   2.718914673173837
SER 153(A)N(donor) ------ O2(acceptor)   3.078475109530692
SER 153(A)OG(donor) ------ O2(acceptor)   3.162595611202927


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [41]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol

def parse_pdb(file_path):
    protein_atoms = []
    ligand_atoms = []

    amino_acids = {}
    current_chain = None

    with open(file_path, 'r') as pdb_file:
        for line in pdb_file:
            if line.startswith("ATOM"):
                atom_type = line[12:16].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                chain = line[21]
                residue_number = int(line[22:26])
                residue_name = line[17:20]

                if chain != current_chain:
                    amino_acids = {}
                    current_chain = chain

                if chain not in amino_acids:
                    amino_acids[chain] = {}

                amino_acids[chain][residue_number] = residue_name

                protein_atoms.append((atom_type, x, y, z, chain, residue_number))

            elif line.startswith("HETATM"):
                atom_type = line[12:16].strip()
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])

                ligand_atoms.append((atom_type, x, y, z))

    #print("Parsed protein atoms:", len(protein_atoms))
    #print("Parsed ligand atoms:", len(ligand_atoms))
    #print("Parsed amino acids:", amino_acids)
    #print(ligand_atoms)
    
    return protein_atoms, ligand_atoms, amino_acids


def calculate_distances(protein_atoms, ligand_atoms):
    distances = []

    for protein_atom in protein_atoms:
       # print("Example protein atom:", protein_atom)  # Debug print
        for ligand_atom in ligand_atoms:
           # print("Example ligand atom:", ligand_atom)  # Debug print
            dist = np.sqrt((protein_atom[1] - ligand_atom[1]) ** 2 +
                           (protein_atom[2] - ligand_atom[2]) ** 2 +
                           (protein_atom[3] - ligand_atom[3]) ** 2)
            distances.append((protein_atom[0], ligand_atom[0], dist, protein_atom[4], protein_atom[5]))

    return distances


def sort_distances(distances):
    return sorted(distances, key=lambda x: x[2])
    
def write_to_excel(sorted_distances, output_file='distances.xlsx'):
    df = pd.DataFrame(sorted_distances, columns=['Protein Atom', 'Ligand Atom', 'Distance', 'Chain', 'Residue Number'])
    df.to_excel(output_file, index=False)
    print(f"Distances written to {output_file}")

def find_hydrogen_bonds(sorted_distances):
    hydrogen_bonds = []

    for protein_atom_type, ligand_atom_type, dist, chain, residue_number in sorted_distances:
        if (protein_atom_type[0] in ['N', 'O', 'S'] and ligand_atom_type[0] in ['N', 'O', 'S']) or \
                (ligand_atom_type[0] in ['N', 'O', 'S'] and protein_atom_type[0] in ['N', 'O', 'S']):
            if 2.7 <= dist <= 3.3:  # Adjust the threshold distance for hydrogen bonds
                hydrogen_bonds.append((chain, residue_number, protein_atom_type, ligand_atom_type, dist))

    return hydrogen_bonds

    if hydrogen_bonds:
        print("\nPossible Hydrogen Bond Interactions:")
        for bond in hydrogen_bonds:
            print(bond)
    else:
        print("\nNo possible Hydrogen Bond Interactions found.")

    return hydrogen_bonds

def visualize_ligand_sticks_with_labels(ligand_atoms, hydrogen_bonds):
    # Create a Py3Dmol view
    view = py3Dmol.view(width=800, height=600)

    # Create a PDB file string for ligand atoms
    pdb_string = ""
    for i, (atom_type, x, y, z) in enumerate(ligand_atoms):
        pdb_string += f"ATOM  {i+1:<5} {atom_type:<4} MOL     1    {x:>8.3f}{y:>8.3f}{z:>8.3f}\n"

    # Add ligand atoms from the PDB file string
    view.addModel(pdb_string, "pdb")

    # Set style to stick representation for ligand atoms
    view.setStyle({'stick': {}})

    # Add labels to ligand atoms
    for i, (atom_type, x, y, z) in enumerate(ligand_atoms):
        view.addLabel(atom_type, {'position': {'x': x, 'y': y, 'z': z}})

    # Add light blue spheres for hydrogen atoms
    hydrogen_atoms = [atom for atom in ligand_atoms if atom[0].startswith('H')]
    for i, (atom_type, x, y, z) in enumerate(hydrogen_atoms):
        view.addSphere({'center': {'x': x, 'y': y, 'z': z}, 'radius': 0.3, 'color': 'lightblue'})

     # Add protein atoms as red spheres and label them
    for chain, residue_number, protein_atom_type, ligand_atom_type, dist in hydrogen_bonds:
        amino_acid = amino_acids.get(chain, {}).get(residue_number, "Unknown")
        # Find the coordinates of the protein atom involved in the hydrogen bond
        protein_atom_coords = next((x, y, z) for _, x, y, z, ch, rn in protein_atoms if ch == chain and rn == residue_number)
        # Add red sphere for the protein atom
        view.addSphere({'center': {'x': protein_atom_coords[0], 'y': protein_atom_coords[1], 'z': protein_atom_coords[2]}, 'radius': 0.3, 'color': 'red'})
        # Label the protein atom with amino acid and residue number
        label_text = f"{amino_acid} {residue_number}({chain})"
        view.addLabel(label_text, {'position': {'x': protein_atom_coords[0], 'y': protein_atom_coords[1], 'z': protein_atom_coords[2]}})
         
    # Add yellow dotted lines between ligand atoms involved in hydrogen bonds
    for chain, residue_number, protein_atom_type, ligand_atom_type, dist in hydrogen_bonds:
        # Find the coordinates of the ligand atom involved in the hydrogen bond
        ligand_atom_coords = None
        for atom in ligand_atoms:
            if atom[0] == ligand_atom_type:
                ligand_atom_coords = atom[1:4]
                break
        if ligand_atom_coords:
            # Find the coordinates of the protein atom involved in the hydrogen bond
            protein_atom_coords = None
            for _, x, y, z, ch, rn in protein_atoms:
                if ch == chain and rn == residue_number:
                    protein_atom_coords = (x, y, z)
                    break
            if protein_atom_coords:
                # Add yellow dotted line between ligand atom and protein atom
        
               # Define the number of segments for the line
                num_segments = 10
                
                # Calculate the length of each segment
                segment_length = dist / num_segments
                
                # Add yellow dotted line between ligand atom and protein atom with increased thickness using cylinder shape
                for i in range(num_segments):
                    start_pos = {'x': ligand_atom_coords[0] + (protein_atom_coords[0] - ligand_atom_coords[0]) * i / num_segments,
                                 'y': ligand_atom_coords[1] + (protein_atom_coords[1] - ligand_atom_coords[1]) * i / num_segments,
                                 'z': ligand_atom_coords[2] + (protein_atom_coords[2] - ligand_atom_coords[2]) * i / num_segments}
                    end_pos = {'x': ligand_atom_coords[0] + (protein_atom_coords[0] - ligand_atom_coords[0]) * (i + 0.5) / num_segments,
                               'y': ligand_atom_coords[1] + (protein_atom_coords[1] - ligand_atom_coords[1]) * (i + 0.5) / num_segments,
                               'z': ligand_atom_coords[2] + (protein_atom_coords[2] - ligand_atom_coords[2]) * (i + 0.5) / num_segments}
                    view.addCylinder({'start': start_pos, 'end': end_pos, 'radius': 0.07, 'color': 'yellow'})


    # Set background color to black
    view.setBackgroundColor('black')

    # Zoom to fit the structure
    view.zoomTo()

    # Show the 3D view
    view.show()





pdb_file_path = "E:\\THESIS\\BACK_END\\4MS4-20240302T114112Z-001\\4MS4\\2.pdb"

protein_atoms, ligand_atoms, amino_acids = parse_pdb(pdb_file_path)
distances = calculate_distances(protein_atoms, ligand_atoms)
sorted_distances = sort_distances(distances)


# Write sorted distances to an Excel file
write_to_excel(sorted_distances, output_file='sorted_distances.xlsx')


# Finding and printing possible hydrogen bonds
hydrogen_bonds = find_hydrogen_bonds(sorted_distances)
if hydrogen_bonds:
    print("\nPossible Hydrogen Bond Interactions:")
    for chain, residue_number, protein_atom_type, ligand_atom_type, dist in hydrogen_bonds:
        amino_acid = amino_acids.get(chain, {}).get(residue_number, "Unknown")
        print(f"{amino_acid} {residue_number}({chain}){protein_atom_type}(donor) ------ {ligand_atom_type}(acceptor)   {dist}")
else:
    print("\nNo possible Hydrogen Bond Interactions found.")


# Call visualize_ligand_sticks_with_labels with ligand_atoms, hydrogen_bonds, protein_atoms, and amino_acids
visualize_ligand_sticks_with_labels(ligand_atoms, hydrogen_bonds)


Distances written to sorted_distances.xlsx

Possible Hydrogen Bond Interactions:
SER 130(A)OG(donor) ------ O1(acceptor)   2.761596639627155
GLU 349(A)OE2(donor) ------ N1(acceptor)   2.7921543653601977
SER 130(A)N(donor) ------ O1(acceptor)   2.9971674961536587
HIS 170(A)NE2(donor) ------ N1(acceptor)   3.1169967917853225
SER 153(A)OG(donor) ------ O2(acceptor)   3.1392843133427744
TRP 278(A)NE1(donor) ------ N1(acceptor)   3.2784769329675014


3Dmol.js failed to load for some reason. Please check your browser console for error messages.